#### This program aims to recreate the plots and diagrams in Section 5.4 of Physics by Computer by Kinzel and Reents.

## 5.4 Polymer Chain

A random walk (a randomly generated path) can be used to model the essential properties and structures associated to polymer chains. We define our random walk on a lattice for convenience.

If N is the number of steps and $R_N$ is the distance between the two ends of the walk, then

$$ N = \frac{\langle R_N^2 \rangle}{a^2} $$

where $\langle ... \rangle$ is an average over all random walks and a is the length of the individual chain links. Letting $L = \langle R_N^2 \rangle $, then

$$ N \propto L^D $$

with the dimension D=2. The mean end-to-end distance L of a SAW will be larger. Polymer will not be as compact. For large values of N, SAW obeys the law above, but D now depends on the spatial dimension d using a mean-field approximation

$$ D = \frac{d+2}{3} $$

The probability of generating a particular chain configuration with N links is

$$ W_N = \prod_{i=1}^{N} \frac{1}{Z_i} $$

where $Z_i$ is the number of sites next to its head. However, the cahins with many contact points (small $Z_i$) are assigned a high probility. One has to assign the statistical weight to each link:

$$ W_N \cdot \prod_{i=1}^{N} \frac{1}{Z_i} = 1$$

$$ L^2 = \langle R_N^2 \rangle = \frac{\sum_{l}R_{N,l}^2\prod_{i=1}^{N}Z_{i,l}}{\sum_{l}\prod_{i=1}^{N}Z_{i,l}} $$

Time evolution of $p_l(t)$

$$\Delta p_l \equiv p_l(t+1) - p_l(t) = \sum_{k=1}^{N}[W(k \to l)p_k - W(l \to k)p_l]$$

Looking at $\Delta p_l = 0$ (stationary state),

$$\sum_{k=1}^{N} W(k \to l)p_k = p_l$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

### Get the adjacent neighbours of the head of the chain

In [2]:
def neighbours(choices):
    indices = []
    for n in choices:
        if n == x_chain[-2] or n == y_chain[-2]:
            indices.append(choices.index(n))
            choices.remove(n) #remove from the choices the cell right before the head of the chain
            
    return choices, indices

### Randomly choose an adjacent empty cell to the head 

In [3]:
def choice(head):
    checker = np.arange(4) 
    pair_val = 2*[head[1]] + 2*[head[0]]
    adjacent, indices = neighbours([head[0]+1,head[0]-1,head[1]+1,head[1]-1])
    checker = np.delete(checker, indices)
    pair_val = np.delete(pair_val, indices)
    step = np.random.randint(len(adjacent))
    
    return adjacent[step], pair_val[step], checker[step]

### Plot each step of the algorithm onto a lattice to visualize the movement of the polymer chain

In [6]:
def images(x_chain,y_chain,head,tail,L):
    lattice = np.zeros((40,40))
    
    for i in range(len(x_chain)):
        y, x = x_chain[i], y_chain[i]
        lattice[x][y] = 2
            
    lattice[tail[1]][tail[0]] = 1 #tail
    lattice[head[1]][head[0]] = 3 #head
    
    im = ax.matshow(lattice, cmap='Purples', animated=True)
    ax.axis('off')
    t = ax.annotate("L = " + str(L), (1,0))

    return [im, t]

### The head and tail of the chain is flipped

In [7]:
def flip(head,tail,x_chain,y_chain):
    head, tail = tail, head
    x_chain = np.flip(x_chain).tolist()
    y_chain = np.flip(y_chain).tolist()
    
    return head,tail,x_chain,y_chain

### Move the whole chain forward into the new empty cell

In [8]:
def move(head,x_chain,y_chain,checker):
    
    if checker < 2: #chain moves along the x-axis
        x_chain.append(new_head)
        y_chain.append(head[1])
        head[0] = new_head
    elif checker > 1: #chain moves along the y-axis
        y_chain.append(new_head)
        x_chain.append(head[0])
        head[1] = new_head
    
    x_chain.pop(0)
    y_chain.pop(0)
    
    return head,x_chain,y_chain

The main body of the program follows these steps for the reptation of a polymer chain with $N$ elements.
1. Start with a configurtion on the lattice, labeling the ends of the chain as head and tail respectively.
2. Remove the last monomer at the tail end and randomly select one of the sites adjacent to the head (three possibilities on the square lattice). 
3. If the site is free, add a new head monomer there. If it is occupied, restore the old configurtion, interchange the labels head and tail, and take the otherwise unmodified configuration into account in calculating averages.
4. Iterate steps 2 and 3.

In [13]:
#declare how long and where the chain is in the lattice
x_chain = np.arange(10,30, dtype=int).tolist()
y_chain = len(x_chain)*[20]

#set the coordinates of the head and tail
head = [x_chain[-1],y_chain[-1]]
tail = [x_chain[0],y_chain[0]]

fig = plt.figure()
ax = fig.add_subplot(111)
ims = []
N = 200 #number of steps
L = 0

for i in range(N):
    #choose a random adjacent cell (3 possibilities) to the head
    new_head, pair_val, checker = choice(head)
    Flip = False
    
    #calculate the mean length of the polymer chain
    L += np.sqrt((head[0]-tail[0])**2 + (head[1]-tail[1])**2)
    
    for n in range(len(x_chain)):
        x, y = x_chain[n], y_chain[n]
        
        #check if the new position of the head is empty
        #if it's not, interchange the head and the tail
        if (checker < 2 and x == new_head and pair_val == y) or (checker > 1 and y == new_head and pair_val == x) or new_head == 40 or new_head < 0:
            head,tail,x_chain,y_chain = flip(head,tail,x_chain,y_chain) #flip head and tail
            Flip = True
            break 
            
    #if the new position of the head is empty, move polymer chain forward
    if Flip == False:
        head,x_chain,y_chain = move(head,x_chain,y_chain,checker)
   
    #set the new tail
    tail = [x_chain[0],y_chain[0]]
    ims.append(images(x_chain,y_chain,head,tail,L/(i+1)))
    
plt.close()

### Animation of the polymer chain for each time step
The polymer chain behaves similar to the mobile phone game "Snake." The chain moves along the lattice. Just like the snake, if it "eats" itself, the chain flips. The tail becomes the head, and the head becomes the tail. This random walk is repeated $N$ times. 

In [14]:
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                repeat_delay=1000)
HTML(ani.to_jshtml(fps=7))

The dark purple square is the head, while the light purple square is the tail. The mean length $L$ is also displayed for each timestep.